In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# ─────── CONFIG ───────
IMG_HEIGHT, IMG_WIDTH = 50, 200
MAX_LABEL_LEN = 10
CHARS = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"

In [ ]:
CHAR_TO_IDX = {c: i + 1 for i, c in enumerate(CHARS)}  # 1-indexed (CTC blank = 0)
IDX_TO_CHAR = {0: ''}  # blank
IDX_TO_CHAR.update({i + 1: c for i, c in enumerate(CHARS)})

In [ ]:
CSV_PATH  = r"C:\Users\tejas\Downloads\archive (1)\Licplatesrecognition_train.csv"
IMAGE_DIR = r"C:\Users\tejas\Downloads\archive (1)\Licplatesrecognition_train\license_plates_recognition_train"

In [ ]:
# ─────── LOAD DATA ───────
print("Loading dataset …")
df = pd.read_csv(CSV_PATH)
images, texts = [], []

In [ ]:
for _, row in df.iterrows():
    img_path = os.path.join(IMAGE_DIR, row["img_id"])
    text = row["text"].strip()

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img = img.astype("float32") / 255.0

    images.append(np.expand_dims(img, axis=-1))
    texts.append(text)

In [ ]:
X = np.array(images, dtype="float32")

In [ ]:
# ─────── LABEL ENCODING ───────
def encode_and_pad_labels(texts, maxlen, char_to_idx):
    num_classes = len(char_to_idx) + 1
    safe_labels = []
    for t in texts:
        cleaned = [char_to_idx[c] for c in t if c in char_to_idx]
        deduped, prev = [], None
        for c in cleaned:
            if c != prev:
                deduped.append(c)
                prev = c
        deduped = [i for i in deduped if 0 < i < num_classes - 1][:maxlen]
        padded  = deduped + [0] * (maxlen - len(deduped))
        safe_labels.append(padded)
    return np.array(safe_labels, dtype=np.int32)

In [ ]:
Y = encode_and_pad_labels(texts, MAX_LABEL_LEN, CHAR_TO_IDX)
assert np.max(Y) < len(CHAR_TO_IDX) + 1, "Labels contain invalid indices!"
print("Dataset shape:", X.shape, Y.shape)

In [ ]:
# ─────── MODEL ───────
def build_crnn_model():
    inp_img = layers.Input((IMG_HEIGHT, IMG_WIDTH, 1), name="input_image")
    x = layers.Conv2D(32, 3, padding="same", activation="relu")(inp_img)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Reshape((IMG_WIDTH // 4, (IMG_HEIGHT // 4) * 64))(x)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    y_pred = layers.Dense(len(CHAR_TO_IDX) + 1, activation="softmax", name="y_pred")(x)
    return inp_img, y_pred

In [ ]:
inp_img, y_pred = build_crnn_model()

In [ ]:
# ─────── CTC LOSS ───────
labels_in = layers.Input(shape=(MAX_LABEL_LEN,), name="labels")
input_len = layers.Input(shape=(1,), name="input_len")
label_len = layers.Input(shape=(1,), name="label_len")

In [ ]:
ctc_output = layers.Lambda(
    lambda x: K.ctc_batch_cost(
        tf.cast(x[1], dtype='int32'),  # labels
        x[0],                          # y_pred
        x[2],                          # input_len
        x[3]                           # label_len
    ),
    output_shape=(1,),
    name="ctc"
)([y_pred, labels_in, input_len, label_len])

In [ ]:
model = models.Model(
    inputs=[inp_img, labels_in, input_len, label_len],
    outputs=ctc_output
)
model.compile(optimizer="adam", loss={"ctc": lambda y_true, y_pred: y_pred})
model.summary()

In [ ]:
# ─────── TRAINING ───────
X_tr, X_val, Y_tr, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
train_dict = {
    "input_image": X_tr,
    "labels": Y_tr,
    "input_len": np.ones((X_tr.shape[0], 1)) * (IMG_WIDTH // 4),
    "label_len": np.array([[np.count_nonzero(y)] for y in Y_tr])
}
val_dict = {
    "input_image": X_val,
    "labels": Y_val,
    "input_len": np.ones((X_val.shape[0], 1)) * (IMG_WIDTH // 4),
    "label_len": np.array([[np.count_nonzero(y)] for y in Y_val])
}
dummy_tr = np.zeros((X_tr.shape[0], 1))
dummy_val = np.zeros((X_val.shape[0], 1))

In [ ]:
print("Training model …")
model.fit(train_dict, dummy_tr, validation_data=(val_dict, dummy_val), epochs=50, batch_size=32)

In [ ]:
# ─────── SAVE INFERENCE MODEL ───────
predict_model = models.Model(inp_img, y_pred)
predict_model.save("predict_crnn_model.keras")
print("[ok] Inference model saved as predict_crnn_model.keras")

In [ ]:
# ─────── DECODING FUNCTION (IMPROVED) ───────
def decode_batch(images, beam_w=10):
    preds = predict_model.predict(images)
    print("Preds shape:", preds.shape)
    print("Sample prediction argmax:", np.argmax(preds[0], axis=-1))

    decoded, _ = K.ctc_decode(
        preds,
        input_length=np.ones(preds.shape[0]) * preds.shape[1],
        greedy=False,
        beam_width=beam_w,
        top_paths=1
    )

    texts = []
    for i, seq in enumerate(decoded[0].numpy()):
        print(f"Decoded indices ({i}):", seq)
        chars = [IDX_TO_CHAR.get(idx, '') for idx in seq if idx > 0]
        result = ''.join(chars)
        print(f"Decoded string ({i}):", result)
        texts.append(result)
    return texts

In [ ]:
# ─────── SANITY CHECK ───────
print("\nDecoding 10 validation samples …")
sample_imgs = X_val[:10]
decoded_texts = decode_batch(sample_imgs, beam_w=10)

In [ ]:
for i, txt in enumerate(decoded_texts):
    gt = "".join([IDX_TO_CHAR.get(c, '') for c in Y_val[i] if c > 0])
    print(f"GT: {gt:<10} | Pred: {txt}")

    plt.imshow(sample_imgs[i].squeeze(), cmap="gray")
    plt.title(f"GT: {gt} | Pred: {txt}")
    plt.axis('off')
    plt.show()